# Sucrose infusions with high fat data

In [87]:
# -- import packages --
import pandas as pd
import numpy as np 
import time
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, KFold,  cross_val_score
from sklearn.model_selection import train_test_split
from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)
from numpy import mean,std



## Data Load 

In [88]:
# sucrose infusions csv dataframe, locally set path
path="C:\\Users\\19802\\Documents\\nibl\\pilot_connectivity\\mouse_data\\Sucrose_infusions_full_df.csv" 
data=pd.read_csv(path) # read in as dataframe

## Prepare Data  


**Dummy variables**  
We use pandas package `get_dummies()` , passing the dataframe and the columns to convert: "MouseId" and "Treatment".   


In [89]:
data['MouseId'].unique(), data['Treatment'].unique()

(array(['E_A1', 'E_A3', 'E_A7', 'E_A8'], dtype=object),
 array(['Post_HFD', 'Post_removal', 'Pre_HFD'], dtype=object))

In [90]:
data=pd.get_dummies(data, columns=["MouseId"]) # convert to dummy/indicator variables
data=pd.get_dummies(data, columns=["Treatment"]) # convert to dummy variables

In [91]:
data.filter(like="MouseId", axis=1).head() , data.filter(like="Treatment", axis=1).head() # view the new dummy/indicator variables

(   MouseId_E_A1  MouseId_E_A3  MouseId_E_A7  MouseId_E_A8
 0             1             0             0             0
 1             1             0             0             0
 2             1             0             0             0
 3             1             0             0             0
 4             1             0             0             0,
    Treatment_Post_HFD  Treatment_Post_removal  Treatment_Pre_HFD
 0                   1                       0                  0
 1                   1                       0                  0
 2                   1                       0                  0
 3                   1                       0                  0
 4                   1                       0                  0)

---

## Prepare Y

In [92]:
# -- Setup the target variables  --

Y = data.loc[:, "Reward"] # set copy of target variable to Y 

# make condition masks for Water vs. Sucrose %5 and Water vs. Sucorse vs 15%
s5_cond_mask=Y.isin(['Water', "Sucrose_5"])
s15_cond_mask=Y.isin(['Water', "Sucrose_15"])

# filter data for target variabes w/ condition masks
y_s5=Y[s5_cond_mask]
y_s15=Y[s15_cond_mask]


# prepare feature dataframe with filtered masks
s5_data = data[s5_cond_mask]
s15_data = data[s15_cond_mask]

y_s5.unique(), y_s15.unique()


(array(['Water', 'Sucrose_5'], dtype=object),
 array(['Water', 'Sucrose_15'], dtype=object))

**Encode target variables** 

In [93]:
# -- label encoder on our data -- 
le = LabelEncoder() # initialize encoder obj
y_s5_enc = le.fit_transform(y_s5) # fit and transform the 5% data
y_s15_enc = le.fit_transform(y_s15) # fit and transform the 15% data

y_s5_enc.shape, y_s15_enc.shape 

((12291,), (12342,))

## Model Setup
SVM classifier with a linear kernel and RBF kernel, with K-fold cross nested validation.

In [94]:

# configure the cross-validation procedure
cv_inner = KFold(n_splits=10, shuffle=True, random_state=1)
cv_outer = KFold(n_splits=5, shuffle=True, random_state=1)

# define the classifiers
svc_linear = SVC(kernel='linear', max_iter=10000)
svc_rbf = SVC(kernel='rbf', max_iter=10000)


# define the hyperparameter grid
C_range =  np.logspace(-3, 2, 6)
gamma_range = np.logspace(-3, 2, 6)

param_grid_linear = dict(C=C_range)
param_grid_rbf = dict(gamma=gamma_range, C=C_range)

# set grid object
grid_linear = GridSearchCV(svc_linear, param_grid_linear, scoring='accuracy',  n_jobs=3, cv=cv_inner)
grid_rbf = GridSearchCV(svc_rbf,param_grid_rbf , scoring='accuracy', n_jobs=2, cv=cv_inner)

grid_linear, grid_rbf

(GridSearchCV(cv=KFold(n_splits=10, random_state=1, shuffle=True),
              estimator=SVC(kernel='linear', max_iter=10000), n_jobs=3,
              param_grid={'C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02])},
              scoring='accuracy'),
 GridSearchCV(cv=KFold(n_splits=10, random_state=1, shuffle=True),
              estimator=SVC(max_iter=10000), n_jobs=2,
              param_grid={'C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02]),
                          'gamma': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02])},
              scoring='accuracy'))

In [80]:
# initialize scaler object
scaler = StandardScaler()

---

## Low Gamma

### Low Gamma BLA

In [81]:
# -- BlA  -- 
s5_lgbla_data=s5_data[["BLA_low_gamma",  'MouseId_E_A1', 'MouseId_E_A3', 'MouseId_E_A7', 'MouseId_E_A8',
                   'Treatment_Post_HFD', 'Treatment_Post_removal','Treatment_Pre_HFD']] #'MouseId','Treatment']] 
s15_lgbla_data=s15_data[["BLA_low_gamma",'MouseId_E_A1', 'MouseId_E_A3', 'MouseId_E_A7', 'MouseId_E_A8',
                   'Treatment_Post_HFD', 'Treatment_Post_removal', 'Treatment_Pre_HFD']] #'MouseId','Treatment']] 

X_s5_lgbla = scaler.fit_transform(s5_lgbla_data) # fit and transform 
X_s15_lgbla = scaler.fit_transform(s15_lgbla_data) # fit and transform

# Making train set for Nested CV and test set for final model evaluation
X_train_s5_lgbla, X_test_s5_lgbla, y_train_s5_lgbla, y_test_s15_lgbla = train_test_split(X_s5_lgbla, y_s5_enc,
                                                    train_size=0.8, 
                                                    test_size=0.2,
                                                    random_state=1)
                                                                                         
                                                    #stratify= y_s15_enc)
X_train_s15_lgbla, X_test_s15_lgbla, y_train_s15_lgbla, y_test_s15_lgbla = train_test_split(X_s15_lgbla, y_s15_enc,
                                                    train_size=0.8, 
                                                    test_size=0.2,
                                                    random_state=1)

---

> LINEAR Kernel

In [ ]:
# execute the nested cross-validation
start_time = time.time()
scores_s5_lgbla_linear = cross_val_score(grid_linear, X_train_s5_lgbla, y_train_s5_lgbla, scoring='accuracy', cv=cv_outer, n_jobs=3) #linear kernel
print("[INFO] --- %s seconds ---" % (time.time() - start_time))
# execute the nested cross-validation
start_time = time.time()
scores_s15_lgbla_linear = cross_val_score(grid_linear, X_train_s15_lgbla, y_train_s15_lgbla, scoring='accuracy', cv=cv_outer, n_jobs=3)
print("[INFO]--- %s seconds ---" % (time.time() - start_time))



In [49]:
print('[INFO] LOW GAMMA BLA, SVM w/ Linear kernel, Accuracy: %.3f (%.3f)' % (mean(scores_s5_lgbla_linear), std(scores_s5_lgbla_linear)))
print('[INFO] LOW GAMMA BLA, SVM w/ Linear kernel, Accuracy: %.3f (%.3f)' % (mean(scores_s15_lgbla_linear), std(scores_s15_lgbla_linear)))


[INFO] LOW GAMMA BLA, SVM w/ Linear kernel, Accuracy: 0.502 (0.014)
[INFO] LOW GAMMA BLA, SVM w/ Linear kernel, Accuracy: 0.517 (0.019)


> RBF Kernel

In [14]:
# execute the nested cross-validation
start_time = time.time()
scores_s15_lgbla_rbf = cross_val_score(grid_rbf, X_s15_lgbla, y_s15_enc,
                                       scoring='accuracy', cv=cv_outer, n_jobs=3)
print("[INFO] --- %s seconds ---" % (time.time() - start_time))



[INFO] --- 1293.4406316280365 seconds ---


In [22]:
print('[INFO] LOW GAMMA BLA, SVM w/ RBF kernel, Accuracy: %.3f (%.3f)' % (mean(scores_s15_lgbla_rbf), std(scores_s15_lgbla_rbf)))


[INFO] LOW GAMMA BLA, SVM w/ RBF kernel, Accuracy: 0.574 (0.009)


---

### Low Gamma PFC

In [17]:
# -- PFC  -- 
s5_lgpfc_data=s5_data[["PFC_low_gamma",  'MouseId_E_A1', 'MouseId_E_A3', 'MouseId_E_A7', 'MouseId_E_A8',
                   'Treatment_Post_HFD', 'Treatment_Post_removal','Treatment_Pre_HFD']] #'MouseId','Treatment']] 
s15_lgpfc_data=s15_data[["PFC_low_gamma",'MouseId_E_A1', 'MouseId_E_A3', 'MouseId_E_A7', 'MouseId_E_A8',
                   'Treatment_Post_HFD', 'Treatment_Post_removal', 'Treatment_Pre_HFD']] #'MouseId','Treatment']] 

X_s5_lgpfc = scaler.fit_transform(s5_lgpfc_data) # fit and transform 
X_s15_lgpfc = scaler.fit_transform(s15_lgpfc_data) # fit and transform

# Making train set for Nested CV and test set for final model evaluation
X_train_s5_lgpfc, X_test_s5_lgpfc, y_train_s5_lgpfc, y_test_s15_lgpfc = train_test_split(X_s5_lgpfc, y_s5_enc,
                                                    train_size=0.8, 
                                                    test_size=0.2,
                                                    random_state=1, shuffle=True)
                                                                                         
                                                    #stratify= y_s15_enc)
X_train_s15_lgpfc, X_test_s15_lgpfc, y_train_s15_lgpfc, y_test_s15_lgpfc = train_test_split(X_s15_lgpfc, y_s15_enc,
                                                    train_size=0.8, 
                                                    test_size=0.2,
                                                    random_state=1,
                                                    shuffle=True)

> Linear Kernel

In [50]:
# execute the nested cross-validation
start_time = time.time()
scores_s5_lgpfc_linear = cross_val_score(grid_linear, X_train_s5_lgpfc, y_train_s5_lgpfc, scoring='accuracy', cv=cv_outer, n_jobs=3) #linear kernel
print("[INFO] --- %s seconds ---" % (time.time() - start_time))
# execute the nested cross-validation
start_time = time.time()
scores_s15_lgpfc_linear = cross_val_score(grid_linear, X_train_s15_lgpfc, y_train_s15_lgpfc, scoring='accuracy', cv=cv_outer, n_jobs=3)
print("[INFO] --- %s seconds ---" % (time.time() - start_time))



[INFO] --- 331.405663728714 seconds ---
[INFO] --- 326.7904350757599 seconds ---


In [52]:
print('[INFO] LOW GAMMA PFC, SVM w/ Linear kernel, Accuracy: %.3f (%.3f)' % (mean(scores_s5_lgpfc_linear), std(scores_s5_lgpfc_linear)))
print('[INFO] LOW GAMMA PFC, SVM w/ Linear kernel, Accuracy: %.3f (%.3f)' % (mean(scores_s15_lgpfc_linear), std(scores_s15_lgpfc_linear)))


[INFO] LOW GAMMA PFC, SVM w/ Linear kernel, Accuracy: 0.502 (0.014)
[INFO] LOW GAMMA PFC, SVM w/ Linear kernel, Accuracy: 0.517 (0.019)


---

In [18]:
# execute the nested cross-validation
start_time = time.time()
scores_s15_lgpfc_rbf = cross_val_score(grid_rbf, X_s15_lgpfc, y_s15_enc,
                                       scoring='accuracy', cv=cv_outer, n_jobs=3)
print("[INFO] --- %s seconds ---" % (time.time() - start_time))



[INFO] --- 1304.6334519386292 seconds ---


In [19]:
print('[INFO] LOW GAMMA PFC, SVM w/ RBF Linear kernel, Accuracy: %.3f (%.3f)' % (mean(scores_s15_lgpfc_rbf), std(scores_s15_lgpfc_rbf)))


[INFO] LOW GAMMA PFC, SVM w/ RBF Linear kernel, Accuracy: 0.593 (0.005)


---

### Low Gamma NAc

In [82]:
# -- PFC  -- 
s5_lgnac_data=s5_data[["NAc_low_gamma",  'MouseId_E_A1', 'MouseId_E_A3', 'MouseId_E_A7', 'MouseId_E_A8',
                   'Treatment_Post_HFD', 'Treatment_Post_removal','Treatment_Pre_HFD']] #'MouseId','Treatment']] 
s15_lgnac_data=s15_data[['NAc_low_gamma','MouseId_E_A1', 'MouseId_E_A3', 'MouseId_E_A7', 'MouseId_E_A8',
                   'Treatment_Post_HFD', 'Treatment_Post_removal', 'Treatment_Pre_HFD']] #'MouseId','Treatment']] 

X_s5_lgnac = scaler.fit_transform(s5_lgnac_data) # fit and transform 
X_s15_lgnac = scaler.fit_transform(s15_lgnac_data) # fit and transform

# Making train set for Nested CV and test set for final model evaluation
X_train_s5_lgnac, X_test_s5_lgnac, y_train_s5_lgnac, y_test_s5_lgnac = train_test_split(X_s5_lgnac, y_s5_enc,
                                                    train_size=0.8, 
                                                    test_size=0.2,
                                                    random_state=1, shuffle=True)
                                                                                         
                                                    #stratify= y_s15_enc)
X_train_s15_lgnac, X_test_s15_lgnac, y_train_s15_lgnac, y_test_s15_lgnac = train_test_split(X_s15_lgnac, y_s15_enc,
                                                    train_size=0.8, 
                                                    test_size=0.2,
                                                    random_state=1,
                                                    shuffle=True)

---

In [83]:
# execute the nested cross-validation
start_time = time.time()
scores_s5_lgnac_linear = cross_val_score(grid_linear, X_s5_lgnac, y_s5_enc, scoring='accuracy', cv=cv_outer, n_jobs=3) #linear kernel
print("[INFO] --- %s seconds ---" % (time.time() - start_time))
# execute the nested cross-validation
start_time = time.time()
scores_s15_lgnac_linear = cross_val_score(grid_linear, X_s15_lgnac, y_s15_enc, scoring='accuracy', cv=cv_outer, n_jobs=3)
print("[INFO] --- %s seconds ---" % (time.time() - start_time))



[INFO] --- 103.69754838943481 seconds ---
[INFO] --- 125.03266525268555 seconds ---


In [84]:
print('[INFO] LOW GAMMA NAc, SVM w/ Linear kernel, Accuracy: %.3f (%.3f)' % (mean(scores_s5_lgnac_linear), std(scores_s5_lgnac_linear)))
print('[INFO] LOW GAMMA NAc, SVM w/ Linear kernel, Accuracy: %.3f (%.3f)' % (mean(scores_s15_lgnac_linear), std(scores_s15_lgnac_linear)))


[INFO] LOW GAMMA NAc, SVM w/ Linear kernel, Accuracy: 0.510 (0.016)
[INFO] LOW GAMMA NAc, SVM w/ Linear kernel, Accuracy: 0.503 (0.011)


---

In [85]:
# execute the nested cross-validation
start_time = time.time()
scores_s5_lgnac_rbf = cross_val_score(grid_rbf, X_s5_lgnac, y_s5_enc,
                                       scoring='accuracy', cv=cv_outer, n_jobs=3)
print("[INFO] --- %s seconds ---" % (time.time() - start_time))


# execute the nested cross-validation
start_time = time.time()
scores_s15_lgnac_rbf = cross_val_score(grid_rbf, X_s15_lgnac, y_s15_enc,
                                       scoring='accuracy', cv=cv_outer, n_jobs=3)
print("[INFO] --- %s seconds ---" % (time.time() - start_time))

 
    

[INFO] --- 1288.2046506404877 seconds ---
[INFO] --- 1341.5985696315765 seconds ---


In [86]:
print('[INFO] LOW GAMMA NAc, SVM w/ RBF kernel, Accuracy: %.3f (%.3f)' % (mean(scores_s5_lgnac_rbf), std(scores_s5_lgnac_rbf)))

print('[INFO] LOW GAMMA NAc, SVM w/ RBF kernel, Accuracy: %.3f (%.3f)' % (mean(scores_s15_lgnac_rbf), std(scores_s15_lgnac_rbf)))


[INFO] LOW GAMMA NAc, SVM w/ RBF kernel, Accuracy: 0.594 (0.005)
[INFO] LOW GAMMA NAc, SVM w/ RBF kernel, Accuracy: 0.566 (0.010)


---

### Low Gamma vHip

In [27]:
# -- PFC  -- 
s5_lghip_data=s5_data[['vHip_low_gamma' ,  'MouseId_E_A1', 'MouseId_E_A3', 'MouseId_E_A7', 'MouseId_E_A8',
                   'Treatment_Post_HFD', 'Treatment_Post_removal','Treatment_Pre_HFD']] #'MouseId','Treatment']] 
s15_lghip_data=s15_data[['vHip_low_gamma' ,'MouseId_E_A1', 'MouseId_E_A3', 'MouseId_E_A7', 'MouseId_E_A8',
                   'Treatment_Post_HFD', 'Treatment_Post_removal', 'Treatment_Pre_HFD']] #'MouseId','Treatment']] 

X_s5_lghip = scaler.fit_transform(s5_lghip_data) # fit and transform 
X_s15_lghip = scaler.fit_transform(s15_lghip_data) # fit and transform

# Making train set for Nested CV and test set for final model evaluation
X_train_s5_lghip, X_test_s5_lghip, y_train_s5_lghip, y_test_s5_lghip = train_test_split(X_s5_lghip, y_s5_enc,
                                                    train_size=0.8, 
                                                    test_size=0.2,
                                                    random_state=1, shuffle=True)
                                                                                         
                                                    #stratify= y_s15_enc)
X_train_s15_lghip, X_test_s15_lghip, y_train_s15_lghip, y_test_s15_lghip = train_test_split(X_s15_lghip, y_s15_enc,
                                                    train_size=0.8, 
                                                    test_size=0.2,
                                                    random_state=1,
                                                    shuffle=True)

---

In [67]:
# execute the nested cross-validation
start_time = time.time()
scores_s5_lghip_linear = cross_val_score(grid_linear, X_train_s5_lghip, y_train_s5_lghip, scoring='accuracy', cv=cv_outer, n_jobs=3) #linear kernel
print("[INFO] --- %s seconds ---" % (time.time() - start_time))
# execute the nested cross-validation
start_time = time.time()
scores_s15_lghip_linear = cross_val_score(grid_linear, X_train_s15_lghip, y_train_s15_lghip, scoring='accuracy', cv=cv_outer, n_jobs=3)
print("[INFO] --- %s seconds ---" % (time.time() - start_time))



[INFO] --- 136.87305545806885 seconds ---
[INFO] --- 132.66495084762573 seconds ---


In [68]:
print('[INFO] LOW GAMMA vHIP, SVM w/ Linear kernel, Accuracy: %.3f (%.3f)' % (mean(scores_s5_lghip_linear), std(scores_s5_lghip_linear)))
print('[INFO] LOW GAMMA vHIP, SVM w/ Linear kernel, Accuracy: %.3f (%.3f)' % (mean(scores_s15_lghip_linear), std(scores_s15_lghip_linear)))


[INFO] LOW GAMMA vHIP, SVM w/ Linear kernel, Accuracy: 0.499 (0.010)
[INFO] LOW GAMMA vHIP, SVM w/ Linear kernel, Accuracy: 0.499 (0.013)


In [28]:
# execute the nested cross-validation
start_time = time.time()
scores_s15_lghip_rbf = cross_val_score(grid_rbf, X_s15_lghip, y_s15_enc,
                                       scoring='accuracy', cv=cv_outer, n_jobs=3)
print("[INFO] --- %s seconds ---" % (time.time() - start_time))



[INFO] --- 1297.61492061615 seconds ---


In [29]:
print('[INFO] LOW GAMMA vHIP, SVM w/ RBF kernel, Accuracy: %.3f (%.3f)' % (mean(scores_s15_lghip_rbf), std(scores_s15_lghip_rbf)))


[INFO] LOW GAMMA vHIP, SVM w/ RBF kernel, Accuracy: 0.548 (0.007)


---

## Results

In [31]:

print('[INFO] LOW GAMMA BLA, SVM w/ RBF kernel, Accuracy: %.3f (%.3f)' % (mean(scores_s15_lgbla_rbf), std(scores_s15_lgbla_rbf)))
print('[INFO] LOW GAMMA PFC, SVM w/ RBF  kernel, Accuracy: %.3f (%.3f)' % (mean(scores_s15_lgpfc_rbf), std(scores_s15_lgpfc_rbf)))
print('[INFO] LOW GAMMA NAc, SVM w/ RBF kernel, Accuracy: %.3f (%.3f)' % (mean(scores_s15_lgnac_rbf), std(scores_s15_lgnac_rbf)))
print('[INFO] LOW GAMMA vHIP, SVM w/ RBF kernel, Accuracy: %.3f (%.3f)' % (mean(scores_s15_lghip_rbf), std(scores_s15_lghip_rbf)))


[INFO] LOW GAMMA BLA, SVM w/ RBF kernel, Accuracy: 0.574 (0.009)
[INFO] LOW GAMMA PFC, SVM w/ RBF  kernel, Accuracy: 0.593 (0.005)
[INFO] LOW GAMMA NAc, SVM w/ RBF kernel, Accuracy: 0.566 (0.010)
[INFO] LOW GAMMA vHIP, SVM w/ RBF kernel, Accuracy: 0.548 (0.007)


---

## Theta

> BLA

In [39]:
# -- BlA  -- 
s5_tbla_data=s5_data[["BLA_theta",  'MouseId_E_A1', 'MouseId_E_A3', 'MouseId_E_A7', 'MouseId_E_A8',
                   'Treatment_Post_HFD', 'Treatment_Post_removal','Treatment_Pre_HFD']] #'MouseId','Treatment']] 
s15_tbla_data=s15_data[["BLA_theta",'MouseId_E_A1', 'MouseId_E_A3', 'MouseId_E_A7', 'MouseId_E_A8',
                   'Treatment_Post_HFD', 'Treatment_Post_removal', 'Treatment_Pre_HFD']] #'MouseId','Treatment']] 

X_s5_tbla = scaler.fit_transform(s5_tbla_data) # fit and transform 
X_s15_tbla = scaler.fit_transform(s15_tbla_data) # fit and transform

In [41]:
# execute the nested cross-validation
start_time = time.time()
scores_s5_tbla_linear = cross_val_score(grid_linear, X_s5_tbla, y_s5_enc, scoring='accuracy', cv=cv_outer, n_jobs=3) #linear kernel
print("[INFO] --- %s seconds ---" % (time.time() - start_time))



[INFO] --- 116.67990565299988 seconds ---


In [43]:
print('[INFO] THETA BLA, SVM w/ Linear kernel, Accuracy: %.3f (%.3f)' % (mean(scores_s5_tbla_linear), std(scores_s5_tbla_linear)))


[INFO] THETA BLA, SVM w/ Linear kernel, Accuracy: 0.525 (0.007)


In [42]:
# execute the nested cross-validation
start_time = time.time()
scores_s15_tbla_linear = cross_val_score(grid_linear, X_s15_tbla, y_s15_enc, scoring='accuracy', cv=cv_outer, n_jobs=3)
print("[INFO]--- %s seconds ---" % (time.time() - start_time))


[INFO]--- 114.26261329650879 seconds ---


In [44]:
print('[INFO] THETA BLA, SVM w/ Linear kernel, Accuracy: %.3f (%.3f)' % (mean(scores_s15_tbla_linear), std(scores_s15_tbla_linear)))


[INFO] THETA BLA, SVM w/ Linear kernel, Accuracy: 0.551 (0.012)


In [45]:
# execute the nested cross-validation
start_time = time.time()
scores_s15_tbla_rbf = cross_val_score(grid_rbf, X_s15_tbla, y_s15_enc,
                                       scoring='accuracy', cv=cv_outer, n_jobs=3)
print("[INFO] --- %s seconds ---" % (time.time() - start_time))



[INFO] --- 1351.4043354988098 seconds ---


In [46]:
print('[INFO] LOW GAMMA BLA, SVM w/ RBF kernel, Accuracy: %.3f (%.3f)' % (mean(scores_s15_tbla_rbf), std(scores_s15_tbla_rbf)))


[INFO] LOW GAMMA BLA, SVM w/ RBF kernel, Accuracy: 0.577 (0.008)


> PFC

In [48]:
# -- PFC  -- 
s5_tpfc_data=s5_data[["PFC_theta",  'MouseId_E_A1', 'MouseId_E_A3', 'MouseId_E_A7', 'MouseId_E_A8',
                   'Treatment_Post_HFD', 'Treatment_Post_removal','Treatment_Pre_HFD']] #'MouseId','Treatment']] 
s15_tpfc_data=s15_data[["PFC_theta",'MouseId_E_A1', 'MouseId_E_A3', 'MouseId_E_A7', 'MouseId_E_A8',
                   'Treatment_Post_HFD', 'Treatment_Post_removal', 'Treatment_Pre_HFD']] #'MouseId','Treatment']] 

X_s5_tpfc = scaler.fit_transform(s5_tpfc_data) # fit and transform 
X_s15_tpfc = scaler.fit_transform(s15_tpfc_data) # fit and transform


In [49]:
# execute the nested cross-validation
start_time = time.time()
scores_s5_tpfc_linear = cross_val_score(grid_linear, X_s5_tpfc, y_s5_enc, scoring='accuracy', cv=cv_outer, n_jobs=3) #linear kernel
print("[INFO] --- %s seconds ---" % (time.time() - start_time))



[INFO] --- 114.07128810882568 seconds ---


In [53]:
print('[INFO] THETA PFC, SVM w/ Linear kernel, Accuracy: %.3f (%.3f)' % (mean(scores_s5_tpfc_linear), std(scores_s5_tpfc_linear)))


[INFO] THETA PFC, SVM w/ Linear kernel, Accuracy: 0.501 (0.014)


In [51]:
# execute the nested cross-validation
start_time = time.time()
scores_s15_tpfc_linear = cross_val_score(grid_linear, X_s15_tpfc, y_s15_enc, scoring='accuracy', cv=cv_outer, n_jobs=3) #linear kernel
print("[INFO] --- %s seconds ---" % (time.time() - start_time))



[INFO] --- 132.5967309474945 seconds ---


In [54]:
print('[INFO] THETA PFC, SVM w/ Linear kernel, Accuracy: %.3f (%.3f)' % (mean(scores_s15_tpfc_linear), std(scores_s15_tpfc_linear)))


[INFO] THETA PFC, SVM w/ Linear kernel, Accuracy: 0.518 (0.012)


In [55]:
# execute the nested cross-validation
start_time = time.time()
scores_s15_tpfc_rbf = cross_val_score(grid_rbf, X_s15_tpfc, y_s15_enc,
                                       scoring='accuracy', cv=cv_outer, n_jobs=3)
print("[INFO] --- %s seconds ---" % (time.time() - start_time))



[INFO] --- 1297.7306368350983 seconds ---


In [56]:
print('[INFO]  THETA PFC, SVM w/ RBF kernel, Accuracy: %.3f (%.3f)' % (mean(scores_s15_tpfc_rbf), std(scores_s15_tpfc_rbf)))


[INFO]  THETA PFC, SVM w/ RBF kernel, Accuracy: 0.564 (0.012)


> NAc

In [95]:
# -- PFC  -- 
s5_tnac_data=s5_data[["NAc_theta",  'MouseId_E_A1', 'MouseId_E_A3', 'MouseId_E_A7', 'MouseId_E_A8',
                   'Treatment_Post_HFD', 'Treatment_Post_removal','Treatment_Pre_HFD']] #'MouseId','Treatment']] 
s15_tnac_data=s15_data[['NAc_theta','MouseId_E_A1', 'MouseId_E_A3', 'MouseId_E_A7', 'MouseId_E_A8',
                   'Treatment_Post_HFD', 'Treatment_Post_removal', 'Treatment_Pre_HFD']] #'MouseId','Treatment']] 

X_s5_tnac = scaler.fit_transform(s5_tnac_data) # fit and transform 
X_s15_tnac = scaler.fit_transform(s15_tnac_data) # fit and transform



In [96]:
# execute the nested cross-validation
start_time = time.time()
scores_s5_tnac_linear = cross_val_score(grid_linear, X_s5_tnac, y_s5_enc, scoring='accuracy', cv=cv_outer, n_jobs=3) #linear kernel
print("[INFO] --- %s seconds ---" % (time.time() - start_time))



[INFO] --- 104.51562666893005 seconds ---


In [61]:
print('[INFO] THETA NAc, SVM w/ Linear kernel, Accuracy: %.3f (%.3f)' % (mean(scores_s5_tnac_linear), std(scores_s5_tnac_linear)))


[INFO] THETA NAc, SVM w/ Linear kernel, Accuracy: 0.502 (0.010)


In [60]:
# execute the nested cross-validation
start_time = time.time()
scores_s15_tnac_linear = cross_val_score(grid_linear, X_s15_tnac, y_s15_enc, scoring='accuracy', cv=cv_outer, n_jobs=3) #linear kernel
print("[INFO] --- %s seconds ---" % (time.time() - start_time))



[INFO] --- 131.8620662689209 seconds ---


In [62]:
print('[INFO] THETA NAc, SVM w/ Linear kernel, Accuracy: %.3f (%.3f)' % (mean(scores_s15_tnac_linear), std(scores_s15_tnac_linear)))


[INFO] THETA NAc, SVM w/ Linear kernel, Accuracy: 0.507 (0.014)


In [63]:
# execute the nested cross-validation
start_time = time.time()
scores_s15_tnac_rbf = cross_val_score(grid_rbf, X_s15_tnac, y_s15_enc,
                                       scoring='accuracy', cv=cv_outer, n_jobs=3)
print("[INFO] --- %s seconds ---" % (time.time() - start_time))



[INFO] --- 1364.2642827033997 seconds ---


In [64]:
print('[INFO]  THETA NAc, SVM w/ RBF kernel, Accuracy: %.3f (%.3f)' % (mean(scores_s15_tnac_rbf), std(scores_s15_tnac_rbf)))


[INFO]  THETA NAc, SVM w/ RBF kernel, Accuracy: 0.550 (0.010)


In [97]:
# execute the nested cross-validation
start_time = time.time()
scores_s5_tnac_rbf = cross_val_score(grid_rbf, X_s5_tnac, y_s5_enc,
                                       scoring='accuracy', cv=cv_outer, n_jobs=3)
print("[INFO] --- %s seconds ---" % (time.time() - start_time))



[INFO] --- 1316.278060913086 seconds ---


In [101]:
print('[INFO]  THETA NAc, SVM w/ RBF kernel, Accuracy: %.3f (%.3f)' % (mean(scores_s5_tnac_rbf), std(scores_s5_tnac_rbf)))


[INFO]  THETA NAc, SVM w/ RBF kernel, Accuracy: 0.566 (0.008)


> vHip

In [65]:
# -- PFC  -- 
s5_thip_data=s5_data[['vHip_theta' ,  'MouseId_E_A1', 'MouseId_E_A3', 'MouseId_E_A7', 'MouseId_E_A8',
                   'Treatment_Post_HFD', 'Treatment_Post_removal','Treatment_Pre_HFD']] #'MouseId','Treatment']] 
s15_thip_data=s15_data[['vHip_theta' ,'MouseId_E_A1', 'MouseId_E_A3', 'MouseId_E_A7', 'MouseId_E_A8',
                   'Treatment_Post_HFD', 'Treatment_Post_removal', 'Treatment_Pre_HFD']] #'MouseId','Treatment']] 

X_s5_thip = scaler.fit_transform(s5_thip_data) # fit and transform 
X_s15_thip = scaler.fit_transform(s15_thip_data) # fit and transform

In [66]:
# execute the nested cross-validation
start_time = time.time()
scores_s5_thip_linear = cross_val_score(grid_linear, X_s5_thip, y_s5_enc, scoring='accuracy', cv=cv_outer, n_jobs=3) #linear kernel
print("[INFO] --- %s seconds ---" % (time.time() - start_time))



[INFO] --- 119.55910277366638 seconds ---


In [69]:
print('[INFO] THETA vHip, SVM w/ Linear kernel, Accuracy: %.3f (%.3f)' % (mean(scores_s5_thip_linear), std(scores_s5_thip_linear)))


[INFO] THETA vHip, SVM w/ Linear kernel, Accuracy: 0.502 (0.005)


In [67]:
# execute the nested cross-validation
start_time = time.time()
scores_s15_thip_linear = cross_val_score(grid_linear, X_s15_thip, y_s15_enc, scoring='accuracy', cv=cv_outer, n_jobs=3) #linear kernel
print("[INFO] --- %s seconds ---" % (time.time() - start_time))



[INFO] --- 126.90511131286621 seconds ---


In [68]:
print('[INFO] THETA vHip, SVM w/ Linear kernel, Accuracy: %.3f (%.3f)' % (mean(scores_s15_thip_linear), std(scores_s15_thip_linear)))


[INFO] THETA vHip, SVM w/ Linear kernel, Accuracy: 0.493 (0.007)


In [70]:
# execute the nested cross-validation
start_time = time.time()
scores_s15_thip_rbf = cross_val_score(grid_rbf, X_s15_thip, y_s15_enc,  scoring='accuracy', cv=cv_outer, n_jobs=3)
print("[INFO] --- %s seconds ---" % (time.time() - start_time))



[INFO] --- 1559.5259630680084 seconds ---


In [71]:
print('[INFO]  THETA NAc, SVM w/ RBF kernel, Accuracy: %.3f (%.3f)' % (mean(scores_s15_thip_rbf), std(scores_s15_thip_rbf)))


[INFO]  THETA NAc, SVM w/ RBF kernel, Accuracy: 0.559 (0.007)


---